In [1]:
%load_ext autoreload
%autoreload 2
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import src.sparkclient as scl

Please set env variable SPARK_VERSION


In [3]:
sc = scl.SparkClient("XML_Import")

Ivy Default Cache set to: /Users/igor/.ivy2/cache
The jars for the packages stored in: /Users/igor/.ivy2/jars
:: loading settings :: url = jar:file:/Users/igor/.local/share/virtualenvs/data-studio-exercise-python-sql-x46TW6ms/lib/python3.8/site-packages/pyspark/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml
com.databricks#spark-xml_2.12 added as a dependency
com.amazon.deequ#deequ added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-45ffaaa9-96a8-4254-873a-2bb3f87227ec;1.0
	confs: [default]
	found com.databricks#spark-xml_2.12;0.12.0 in central
	found commons-io#commons-io;2.8.0 in central
	found org.glassfish.jaxb#txw2;2.3.3 in central
	found org.apache.ws.xmlschema#xmlschema-core;2.2.5 in central
	found com.amazon.deequ#deequ;1.2.2-spark-3.0 in central
	found org.scalanlp#breeze_2.12;0.13.2 in central
	found org.scalanlp#breeze-macros_2.12;0.13.2 in central
	found org.scala-lang#scala-reflect;2.12.1 in central
	found com.github.fom

# Load Parquet files

In [4]:
posts = sc.spark.read.parquet("uncommitted/posts.parquet")
posts.printSchema()

root
 |-- AcceptedAnswerId: long (nullable = true)
 |-- AnswerCount: long (nullable = true)
 |-- Body: string (nullable = true)
 |-- ClosedDate: timestamp (nullable = true)
 |-- CommentCount: long (nullable = true)
 |-- CommunityOwnedDate: timestamp (nullable = true)
 |-- ContentLicense: string (nullable = true)
 |-- CreationDate: timestamp (nullable = true)
 |-- FavoriteCount: long (nullable = true)
 |-- Id: long (nullable = true)
 |-- LastActivityDate: timestamp (nullable = true)
 |-- LastEditDate: timestamp (nullable = true)
 |-- LastEditorDisplayName: string (nullable = true)
 |-- LastEditorUserId: long (nullable = true)
 |-- OwnerDisplayName: string (nullable = true)
 |-- OwnerUserId: long (nullable = true)
 |-- ParentId: long (nullable = true)
 |-- PostTypeId: long (nullable = true)
 |-- Score: long (nullable = true)
 |-- Tags: string (nullable = true)
 |-- Title: string (nullable = true)
 |-- ViewCount: long (nullable = true)



In [5]:
tags = sc.spark.read.parquet("uncommitted/tags.parquet")
tags.printSchema()

root
 |-- Count: long (nullable = true)
 |-- ExcerptPostId: long (nullable = true)
 |-- Id: long (nullable = true)
 |-- TagName: string (nullable = true)
 |-- WikiPostId: long (nullable = true)



In [6]:
posts_tags = sc.spark.read.parquet("uncommitted/posts_tags.parquet")
posts_tags.printSchema()

root
 |-- PostId: long (nullable = true)
 |-- TagId: long (nullable = true)
 |-- TagName: string (nullable = true)



# Run Spark SQL
Below is a sample SQL to find most popular tags

In [8]:
posts.registerTempTable("p")
tags.registerTempTable("t")
posts_tags.registerTempTable("pt")

In [21]:
sc.spark.sql("""
SELECT pt.TagName, COUNT(*) AS PostCount
FROM p JOIN pt ON pt.PostId = p.Id
GROUP BY pt.TagName
ORDER BY PostCount DESC
""").toPandas()

,TagName,PostCount
0,neural-networks,2165
1,machine-learning,1908
2,reinforcement-learning,1807
3,deep-learning,1627
4,convolutional-neural-networks,998
...,...,...
986,wake-sleep-algorithm,1
987,reward-hacking,1
988,production-systems,1
989,p-value,1


# Stop Spark

In [3]:
sc.spark.stop()
